# 4th Year Project: Utility Functions
`
Last Modified: 19th February 2020
Author: Ken Yew Piong, Chin Yang Tan, Jing Wei Chan, Ka Shing Liong
Department: MEng Electronic and Electrical Engineering
Institution: University College London
`

```python
# DEVELOPER NOTES
```

---
## Library Import

In [ ]:
import os, math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
from scipy.fftpack import fft
from scipy import signal

---
## 1.0 Global Parameters

In [ ]:
# Global Configuration Dictionary for Plot Customisation
# use pylab.rcParams.update(params) to update settings
import matplotlib.pylab as pylab
params = {'lines.linewidth' : 1,
          'legend.fontsize': 'x-large',
          'figure.figsize': (15, 5),
         'axes.labelsize': 'large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}

# Update plot customisation parameters
# pylab.rcParams.update(params)

# ==============================================
# Initialisation Parameters
# ==============================================
# Filter functions presets
fs = 50 # sampling frequency

---
## 2. Helper Functions
### 2.1 Data Acquisition and Pre-Processing Functions

In [ ]:
def truncate_dataframe(path, file, start1, end1, start2, end2, start3, end3):
    os.chdir(path)
    df = pd.read_csv(file)
    df_trial_1 = df[(df['elapsed (s)'] >= start1) & (df['elapsed (s)'] <= end1)]
    df_trial_2 = df[(df['elapsed (s)'] >= start2) & (df['elapsed (s)'] <= end2)]
    df_trial_3 = df[(df['elapsed (s)'] >= start3) & (df['elapsed (s)'] <= end3)]
    os.chdir('..')
    return df_trial_1, df_trial_2, df_trial_3

### 2.2 Plotting Functions

In [ ]:
def plot_time_3_axes(df, file, savefig):
    """
    :FUNCTION: Plots the time series graph in all 3 axes
    :df: Pandas dataframe of raw acc/gyro data (data after using pd.read_csv)
    :file: CSV filename being worked on
    :savefig: Toggle on whether to save the plot in a .png file
    :rtype: void - plots the time series graph
    """
    # ====================
    # Data Visualization
    # ====================
    select_cols=df.columns[-3:]
    labels=['x-component', 'y-component', 'z-component']
    color_map=['r', 'g', 'b']
    fig, ax = plt.subplots(1,1)
        
    for idx, col in enumerate(select_cols):
        ax.plot(df.loc[:, 'elapsed (s)'], df.loc[:, col], color_map[idx], label=labels[idx])

    ax.set_xlabel('Frequency (Hz)')
    if 'Accelerometer' in file: 
        ax.set_ylabel('Magnitude (g)')
    elif 'Gyroscope' in file: 
        ax.set_ylabel('Magnitude (deg/s)')
    ax.set_title(file)
    ax.legend(loc='best')
    ax.grid()
    
    if savefig == True: 
        fig.savefig(f'{file}.png')
    
def plot_frequency(df, fs, file, savefig):
    """
    :FUNCTION: Plots the FFT graph in all 3 axes and returns computed results as a dataframe
    :df: Pandas dataframe of raw acc/gyro data (data after using pd.read_csv)
    :fs: Sampling frequency
    :file: CSV filename being worked on
    :savefig: Toggle on whether to save the plot in a .png file
    :rtype: returns Pandas dataframe of computed FFT magnitude values for each axes, plots the FFT graph
    """
    # ====================
    # Data Pre-processing
    # ====================
    processed_df = df.copy()
    select_cols = processed_df.columns[-3:]
    col_name = ['FFT magnitude x-axis', 'FFT magnitude y-axis', 'FFT magnitude z-axis']
    result_df = pd.DataFrame()
    
    lgth, num_signal=df.shape
    fqy = np.zeros([lgth, num_signal])
    
    # Perform FFT on data and store in matrix
    for idx, col in enumerate(select_cols): 
        result_df['frequency (Hz)'] = np.arange(int(lgth/2))/(int(lgth/2)/(fs/2))
        fqy[:,idx] = np.abs(fft(df.loc[:, col]))
        result_df[col_name[idx]] = pd.Series(fqy[0:int(lgth/2),idx])
        
    # ====================
    # Data Visualization
    # ====================
    labels=['x-component', 'y-component', 'z-component']
    color_map=['r', 'g', 'b']
    fig, ax = plt.subplots(1,1)
        
    for i in range(3):
        ax.plot(result_df.loc[:, 'frequency (Hz)'], result_df.loc[:, col_name[i]], color_map[i], label=labels[i])

    ax.set_xlabel('Frequency (Hz)')
    if 'Accelerometer' in file: 
        ax.set_ylabel('Magnitude (g)')
    elif 'Gyroscope' in file: 
        ax.set_ylabel('Magnitude (deg/s)')
    ax.set_title(file)
    ax.legend(loc='best')
    ax.grid()
    
    if savefig == True: 
        fig.savefig(f'{file}.png')
        
    return result_df
        
def plot_psd(df, fs, file, savefig):
    """
    :FUNCTION: Plots the power spectral density graph in all 3 axes and returns computed results as a dataframe
    :df: Pandas dataframe of filtered data (data after using filter functions)
    :fs: Sampling frequency
    :file: CSV filename being worked on
    :savefig: Toggle on whether to save the plot in a .png file
    :rtype: returns Pandas dataframe of computed frequency and PSD values for each axes, plots the PSD graph
    """
    # ====================
    # Data Pre-processing
    # ====================
    processed_df = df.copy()
    select_cols = processed_df.columns[-3:]
    psd_col_name = ['psd x-axis', 'psd y-axis', 'psd z-axis']
    result_df = pd.DataFrame()
    
    for idx, col in enumerate(select_cols):
        f_data, psd_data = signal.welch(processed_df.loc[:, col], fs)
        result_df['frequency (Hz)'] = pd.Series(f_data)
        result_df[psd_col_name[idx]] = pd.Series(psd_data)
    
    # ====================
    # Data Visualization
    # ====================
    labels = ['x-component', 'y-component', 'z-component']
    color_map = ['r', 'g', 'b']
    fig, ax = plt.subplots(1,1)

    for i in range(3):
        ax.plot(result_df.loc[:, 'frequency (Hz)'], result_df.loc[:, psd_col_name[i]], color_map[i], label=labels[i])

    ax.set_xlabel('Frequency (Hz)')
    ax.set_ylabel('Power (V^2/Hz)')
    ax.set_title('Power Spectral Density: '+str(file.split('-')[:3]))
    ax.legend(loc='best')
    ax.grid()
    
    if savefig == True: 
        fig.savefig(f'{file}.png')
        
    return result_df
        
def plot_in_3d(df, t = ''):   
    # Initialize figure and 3d projection
    fig = plt.figure(figsize = [10, 10])
    ax = fig.add_subplot(111, projection = '3d')
    # Label axes
    ax.set(xlabel = df.columns[3], ylabel = df.columns[4], zlabel = df.columns[5], title = t) 
    
    # Get datapoints
    x = df.iloc[:, 3]
    y = df.iloc[:, 4]
    z = df.iloc[:, 5]
    
    # Plot
    ax.plot(x, y, z)   

### 2.3 Filter Functions

In [ ]:
def band_pass_filter(df, lowcut, highcut, fs, order, file):
    """
    :FUNCTION: Appends input dataframe with new filtered x,y,z values 
    processed with a band pass filter
    :df: Pandas dataframe of raw acc/gyro data (data after using pd.read_csv)
    :lowcut: Desired lower cut-off frequency 
    :highcut: Desired higher cut-off frequency 
    :fs: Sampling frequency
    :order: Polynomial order of Butterworth filter
    :file: CSV filename being worked on
    :rtype: Processed Pandas dataframe with new filtered x,y,z values appended
    """
    processed_df = df.copy()
    select_cols = processed_df.columns[-3:]
    if 'Accelerometer' in file: 
        col_name = ['filtered x-axis (g)', 'filtered y-axis (g)', 'filtered z-axis (g)']
    elif 'Gyroscope' in file: 
        col_name = ['filtered x-axis (deg/s)', 'filtered y-axis (deg/s)', 'filtered z-axis (deg/s)']
    
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(order, [low, high], btype='band')
    
    for idx, col in enumerate(select_cols):
        processed_df[col_name[idx]] = pd.DataFrame(signal.lfilter(b, a, processed_df.loc[:, col])).values
        
    return processed_df

def high_pass_filter(df, highcut, fs, order, file):
    """
    :FUNCTION: Appends input dataframe with new filtered x,y,z values 
    processed with a high pass filter
    :df: Pandas dataframe of raw acc/gyro data (data after using pd.read_csv)
    :highcut: Desired higher cut-off frequency 
    :fs: Sampling frequency
    :order: Polynomial order of Butterworth filter
    :file: CSV filename being worked on
    :rtype: Processed Pandas dataframe with new filtered x,y,z values appended
    """
    processed_df = df.copy()
    select_cols = processed_df.columns[-3:]
    if 'Accelerometer' in file: 
        col_name = ['filtered x-axis (g)', 'filtered y-axis (g)', 'filtered z-axis (g)']
    elif 'Gyroscope' in file: 
        col_name = ['filtered x-axis (deg/s)', 'filtered y-axis (deg/s)', 'filtered z-axis (deg/s)']
    
    nyq = 0.5 * fs
    high = highcut / nyq
    b, a = signal.butter(order, high, btype='high', analog=False)
    
    for idx, col in enumerate(select_cols):
        processed_df[col_name[idx]] = pd.DataFrame(signal.lfilter(b, a, processed_df.loc[:, col])).values
        
    return processed_df

def low_pass_filter(df, lowcut, fs, order, file):
    """
    :FUNCTION: Appends input dataframe with new filtered x,y,z values 
    processed with a low pass filter
    :df: Pandas dataframe of raw acc/gyro data (data after using pd.read_csv)
    :lowcut: Desired lower cut-off frequency 
    :fs: Sampling frequency
    :order: Polynomial order of Butterworth filter
    :file: CSV filename being worked on
    :rtype: Processed Pandas dataframe with new filtered x,y,z values appended
    """
    processed_df = df.copy()
    select_cols = processed_df.columns[-3:]
    if 'Accelerometer' in file: 
        col_name = ['filtered x-axis (g)', 'filtered y-axis (g)', 'filtered z-axis (g)']
    elif 'Gyroscope' in file: 
        col_name = ['filtered x-axis (deg/s)', 'filtered y-axis (deg/s)', 'filtered z-axis (deg/s)']
    
    nyq = 0.5 * fs
    low = lowcut / nyq
    b, a = signal.butter(order, low, btype='low', analog=False)
    
    for idx, col in enumerate(select_cols):
        processed_df[col_name[idx]] = pd.DataFrame(signal.lfilter(b, a, processed_df.loc[:, col])).values
        
    return processed_df

### 2.4 Integration Functions

In [ ]:
# Trapezoidal Integration
def TZ_integration(signal): 
    length = signal.shape
    integral = np.zeros(length)
    
    c = 0
    for idx, _ in enumerate(signal): 
        if idx == 0: 
            integral[idx] = c + signal[idx]
            
        else: 
            integral[idx] = integral[idx - 1] + integral[idx]
    return integral

# Accumulative Integration (based on TZ integration)
def acc_integration(df): 
    select_cols = [3, 4, 5]
    num_rows, num_cols = df.shape
    int_data = np.zeros(df.shape)
    
    for idx, col in enumerate(select_cols): 
        int_data[:, idx] = TZ_integration(data.iloc[:, col])
    return int_data

# Accumulative Integration (based on TZ integration)
def acc_integration_recursive(df): 
    num_rows, num_cols = df.shape
    int_data = np.zeros(df.shape)
    
    for i in range(num_cols): 
        int_data[:, i] = TZ_integration(df[:, i])
    return int_data

### 2.5 Miscellaneous Functions

In [ ]:
# Obtains the directory path of interest
def get_folder_path(foldername):
    """
    :foldername type: string - folder name to search
    :rtype: string - path name of interest
    """
    for dirpath, dirnames, filenames in os.walk(os.getcwd()): 
        for dirname in dirnames: 
            if dirname == foldername: 
                path = os.path.join(dirpath, dirname)
                return path

---
## 3.0 Helper Functions Archived from Term 1
> ##### 3.1 Data Pre-processing of Magnitude and Frequency Rolling Means
> ##### 3.2 Data Visualisation of Magnitude and Frequency Rolling Means
> ##### 3.3 Statistics Collection of Magnitude and Frequency Rolling Means 
> ##### 3.4 Manual Truncation of Processed Data

In [ ]:
# ==============================================
# Initialisation Parameters
# ==============================================
# Rolling mean functions presets
window = 3 # rolling mean window
max_window = 30 # max rolling mean window size

# =================================================================
# 3.1 Data Pre-processing of Magnitude and Frequency Rolling Means
# =================================================================
def process_dataframe(df, file, window, max_window):
    if 'Accelerometer' in file:
        elapsed = df['elapsed (s)']
        # Calculate rolling mean
        x_resultant = df.loc[window-1:,'x-axis (g)']-df['x-axis (g)'].rolling(window).mean()[window-1:]
        y_resultant = df.loc[window-1:,'y-axis (g)']-df['y-axis (g)'].rolling(window).mean()[window-1:]
        z_resultant = df.loc[window-1:,'z-axis (g)']-df['z-axis (g)'].rolling(window).mean()[window-1:]
        # Calculate magnitude and rolling maximum magnitude 
        magnitude = x_resultant**2 + y_resultant**2 + z_resultant**2
        magnitude = (magnitude.apply(math.sqrt))*9.81
        max_magnitude = magnitude.rolling(max_window).max()
        # Calculate frequency and rolling maximum frequency
        freqs, times, spectrogram = signal.spectrogram(magnitude.values, fs=50.0, nperseg=70, noverlap=70-1)
        frequency = pd.Series([freqs[i] for i in np.argmax(spectrogram, 0)])
        max_frequency = frequency.rolling(max_window).max()
        processed_df = pd.DataFrame({'elapsed (s)':elapsed, 
                              'x-axis resultant (g)':x_resultant,
                              'y-axis resultant (g)':y_resultant,
                              'z-axis resultant (g)':z_resultant,
                              'magnitude (g)':magnitude,
                              'rolling maximum magnitude (g)':max_magnitude,
                              'time (s)': pd.Series(times),
                              'frequency (Hz)': frequency,
                              'rolling maximum frequency (Hz)': max_frequency
                             }) 
    elif 'Gyroscope' in file:
        elapsed = df['elapsed (s)']
        # Calculate rolling mean
        x_resultant = df.loc[window-1:,'x-axis (deg/s)']-df['x-axis (deg/s)'].rolling(window).mean()[window-1:]
        y_resultant = df.loc[window-1:,'y-axis (deg/s)']-df['y-axis (deg/s)'].rolling(window).mean()[window-1:]
        z_resultant = df.loc[window-1:,'z-axis (deg/s)']-df['z-axis (deg/s)'].rolling(window).mean()[window-1:]
        # Calculate magnitude and rolling maximum magnitude 
        magnitude = x_resultant**2 + y_resultant**2 + z_resultant**2
        magnitude = (magnitude.apply(math.sqrt))*9.81
        max_magnitude = magnitude.rolling(max_window).max()
        # Calculate frequency and rolling maximum frequency
        freqs, times, spectrogram = signal.spectrogram(magnitude.values, fs=50.0, nperseg=70, noverlap=70-1)
        frequency = pd.Series([freqs[i] for i in np.argmax(spectrogram, 0)])
        max_frequency = frequency.rolling(max_window).max()
        processed_df = pd.DataFrame({'elapsed (s)':elapsed, 
                              'x-axis resultant (deg/s)':x_resultant,
                              'y-axis resultant (deg/s)':y_resultant,
                              'z-axis resultant (deg/s)':z_resultant,
                              'magnitude (deg/s)':magnitude,
                              'rolling maximum magnitude (deg/s)':max_magnitude,
                              'time (s)': pd.Series(times),
                              'frequency (Hz)': frequency,
                              'rolling maximum frequency (Hz)': max_frequency
                             })  
    return processed_df

# =================================================================
# 3.2 Data Visualisation of Magnitude and Frequency Rolling Means
# =================================================================
def plot_magnitude_frequency(df, t = ''):
    if 'Accelerometer' in t:
        ax1 = df.plot(x = 'elapsed (s)', y = ['magnitude (g)', 'rolling maximum magnitude (g)'])
        ax1.set(xlabel = 'elapsed (s)', ylabel = 'magnitude (g)', title = t) 
        ax1.grid(b=True, which='both')
        ax2 = df.plot(x = 'time (s)', y = ['frequency (Hz)', 'rolling maximum frequency (Hz)'])
        ax2.set(xlabel = 'time (s)', ylabel = 'frequency (Hz)', title = t) 
        ax2.grid(b=True, which='both')

    elif 'Gyroscope' in t:
        ax1 = df.plot(x = 'elapsed (s)', y = ['magnitude (deg/s)', 'rolling maximum magnitude (deg/s)'])
        ax1.set(xlabel = 'elapsed (s)', ylabel = 'magnitude (deg/s)', title = t) 
        ax1.grid(b=True, which='both')
        ax2 = df.plot(x = 'time (s)', y = ['frequency (Hz)', 'rolling maximum frequency (Hz)'])
        ax2.set(xlabel = 'time (s)', ylabel = 'frequency (Hz)', title = t) 
        ax2.grid(b=True, which='both')
        
# =================================================================
# 3.3 Statistics Collection of Magnitude and Frequency Rolling Means
# =================================================================
def collect_stats_from_unprocessed_data(df, file, dp):
    if 'Accelerometer' in file: 
        stats_dict = {'elapsed (s)':{}, 'x-axis (g)':{}, 'y-axis (g)':{}, 'z-axis (g)':{}}
    elif 'Gyroscope' in file: 
        stats_dict = {'elapsed (s)':{}, 'x-axis (deg/s)':{}, 'y-axis (deg/s)':{}, 'z-axis (deg/s)':{}}  
    stats_types = ['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']
    df_stats = df.describe()
    for key in stats_dict: 
        for stats_type in stats_types: 
            extracted_stats = df_stats.loc[:, [key]].T
            stats_dict[key][stats_type] = extracted_stats[stats_type][0].round(dp)
    return stats_dict

def collect_stats_from_processed_data(df, file, dp):
    if 'Accelerometer' in file: 
        stats_dict = {'x-axis resultant (g)':{}, 'y-axis resultant (g)':{}, 'z-axis resultant (g)':{}, 'magnitude (g)':{}, 'rolling maximum magnitude (g)':{}, 'frequency (Hz)':{}, 'rolling maximum frequency (Hz)':{}}
    elif 'Gyroscope' in file: 
        stats_dict = {'x-axis resultant (deg/s)':{}, 'y-axis resultant (deg/s)':{}, 'z-axis resultant (deg/s)':{}, 'magnitude (deg/s)':{}, 'rolling maximum magnitude (deg/s)':{}, 'frequency (Hz)':{}, 'rolling maximum frequency (Hz)':{}}
    stats_types = ['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']
    df_stats = df.describe()
    for key in stats_dict: 
        for stats_type in stats_types: 
            extracted_stats = df_stats.loc[:, [key]].T
            stats_dict[key][stats_type] = extracted_stats[stats_type][0].round(dp)
    return stats_dict

def tabulate_stats(file, df, dp, window, max_window): 
    rst = {}
    stats_dict = collect_stats_from_processed_data(df, file, dp)
    if 'Accelerometer' in file:
        mean_magnitude = stats_dict['magnitude (g)']['mean']
        std_magnitude = stats_dict['magnitude (g)']['std']
        mean_frequency = stats_dict['frequency (Hz)']['mean']
        std_frequency = stats_dict['frequency (Hz)']['std']
        rst.update({file:{'magnitude (g)':{'mean': mean_magnitude, 'std': std_magnitude}, 'frequency (Hz)':{'mean': mean_frequency, 'std': std_frequency}}})
    elif 'Gyroscope' in file: 
        mean_magnitude = stats_dict['magnitude (deg/s)']['mean']
        std_magnitude = stats_dict['magnitude (deg/s)']['std']
        mean_frequency = stats_dict['frequency (Hz)']['mean']
        std_frequency = stats_dict['frequency (Hz)']['std']
        rst.update({file:{'magnitude (deg/s)':{'mean': mean_magnitude, 'std': std_magnitude}, 'frequency (Hz)':{'mean': mean_frequency, 'std': std_frequency}}})
    rst_df = pd.concat({key: pd.DataFrame(value).T for key, value in rst.items()}, axis=0)
    os.chdir('..')
    return rst_df

def tabulate_all_stats(path, dp, window, max_window): 
    os.chdir(path)
    directory = os.listdir(path)
    rst = {}
    for file in directory: 
        df = pd.read_csv(file)
        df_processed = process_dataframe(df, file, window, max_window)
        stats_dict = collect_stats_from_processed_data(df_processed, file, dp)
        if 'Accelerometer' in file:
            mean_magnitude = stats_dict['magnitude (g)']['mean']
            std_magnitude = stats_dict['magnitude (g)']['std']
            mean_frequency = stats_dict['frequency (Hz)']['mean']
            std_frequency = stats_dict['frequency (Hz)']['std']
            rst.update({file:{'magnitude (g)':{'mean': mean_magnitude, 'std': std_magnitude}, 'frequency (Hz)':{'mean': mean_frequency, 'std': std_frequency}}})
        elif 'Gyroscope' in file: 
            mean_magnitude = stats_dict['magnitude (deg/s)']['mean']
            std_magnitude = stats_dict['magnitude (deg/s)']['std']
            mean_frequency = stats_dict['frequency (Hz)']['mean']
            std_frequency = stats_dict['frequency (Hz)']['std']
            rst.update({file:{'magnitude (deg/s)':{'mean': mean_magnitude, 'std': std_magnitude}, 'frequency (Hz)':{'mean': mean_frequency, 'std': std_frequency}}})
    rst_df = pd.concat({key: pd.DataFrame(value).T for key, value in rst.items()}, axis=0)
    os.chdir('..')
    return rst_df

# ==============================================
# 3.4 Manual Truncation of Processed Data
# ==============================================
def truncate_processed_dataframe(path, file, start1, end1, start2, end2, start3, end3):
    os.chdir(path)
    df_raw = pd.read_csv(file)
    df = process_dataframe(df_raw, file, 3, 30)
    df_trial_1 = df[(df['elapsed (s)'] >= start1) & (df['elapsed (s)'] <= end1)]
    df_trial_2 = df[(df['elapsed (s)'] >= start2) & (df['elapsed (s)'] <= end2)]
    df_trial_3 = df[(df['elapsed (s)'] >= start3) & (df['elapsed (s)'] <= end3)]
    os.chdir('..')
    return df_trial_1, df_trial_2, df_trial_3

---
## 4.0 Function Calls Archive

In [ ]:
# Uncomment sections below to enable plots of interest
"""
# ==============================================
# Filtered Signals - Data Pre-processing
# ==============================================
median_data=median_filter(df, 155)
lpf_data=freq_filter(df, 155, cutoff/fs)
comb_data=freq_filter_recursive(median_data, 155, cutoff/fs)
"""

"""
# ==============================================
# Low Pass Filter Plots
# ==============================================
plot_time_3_axes(lpf_data, t = "LPF: " + file)
plot_frequency_recursive(lpf_data, fs, t = "LPF: " + file)
"""

"""
# ==============================================
# Median Pass Filter Plots
# ==============================================
plot_time_3_axes(median_data, t = "Median Filter: " + file)
plot_frequency_recursive(median_data, fs, t = "Median Filter: " + file)
"""

"""
# ==============================================
# Combined (Low + Median) Pass Filter Plots
# ==============================================
plot_time_3_axes(comb_data, t = "Combined Filter: " + file)
plot_frequency_recursive(comb_data, fs, t = "Combined Filter: " + file)
"""
"""
# ==============================================
# Integrated Signals Plots
# ==============================================
integrated_data = acc_integration(df)
plot_time_3_axes(integrated_data, t = "Integrated Signal (Velocity): " + file)
"""